### Import Libraries

In [1]:
%reload_ext autoreload

In [2]:
import warnings
warnings.filterwarnings("ignore")
import nltk
import pandas as pd 
import numpy as np
import io
import os
import json
import xgboost as xgb
import lightgbm as lgb
from classes.CustomTokenizer import CustomTokenizer
from classes.Accuracy import Accuracy
from classes.ModelBuilder import ModelBuilder
from sklearn import preprocessing
from sklearn.utils import resample, shuffle
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
pd.set_option('display.max_colwidth', 600)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Select Classifier

In [3]:
classifier = SVC()

### Helper Code

In [4]:
labelEncoders = dict()
def labelEncoder(df, column, params = {}):    
    le = preprocessing.LabelEncoder()        
    le.fit(df[column])    
    column_encoded = le.transform(df[column])
    labelEncoders[column] = le
    return column_encoded

### Get Info from CSV

In [5]:
columns = ['Pregunta', 'Intencion']
df_train = shuffle(pd.read_csv('data/train.csv', usecols=columns, sep='|'))
df_test = shuffle(pd.read_csv('data/test_santander.csv', usecols=['id','Pregunta']))

df_train['Intencion_cat_label'] = df_train['Intencion'].str[4:]
df_train['Intencion_cat_label'] = df_train['Intencion_cat_label'].astype('int32')
df_train['Intencion_encoded'] = labelEncoder(df_train, 'Intencion')

print('Train Shape: ' + str(df_train.shape))
print('Test Shape: ' + str(df_test.shape))

Train Shape: (20104, 4)
Test Shape: (6702, 2)


In [6]:
X = df_train['Pregunta'].values
y = df_train['Intencion_encoded'].values

In [7]:
df_array = pd.DataFrame(columns=df_train.columns)
enableResample = False
if enableResample == True:
    max_value = df_train['Intencion'].value_counts()[0]
    unique_cat = df_train['Intencion'].unique()
    for category in list(unique_cat):
        df_cat_filter = df_train[df_train['Intencion'] == category]        
        if len(df_cat_filter) < max_value:
            # upsample minority
            df_cat_filter = pd.DataFrame(resample(df_cat_filter, replace=True, # sample with replacement
                                     n_samples=max_value # match number in majority class
                                    ) # reproducible results
                                        )                
        df_array = df_array.append(df_cat_filter, ignore_index=True)
    # combine majority and upsampled minority      
    X = df_array['Pregunta'].values
    y = df_array['Intencion_encoded'].values
    print(df_array.Intencion.value_counts())

### train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(np.array(X), y, test_size = 0.20)
y_train = y_train.astype('int')
y_test = y_test.astype('int')

In [ ]:
modelBuilder = ModelBuilder()
optimized_model, model_best_params = modelBuilder.GenerateTrainedModel(classifier, 
                                                                        X_train, 
                                                                        X_test, 
                                                                        y_train, 
                                                                        y_test)

pred = optimized_model.predict(X_test)
#Compute the balanced accuracy
#The balanced accuracy in binary and multiclass classification problems to deal with imbalanced datasets. It is defined as the average of recall obtained on each class.
#The best value is 1 and the worst value is 0 when adjusted=False.
balanced_accuracy_score = Accuracy.get_balanced_accuracy_score(y_test, pred)
accuracy_score = Accuracy.get_accuracy_score(y_test, pred)

Training Model...


### Save Accuracy on CSV file

In [ ]:
columns = ['model_name', 'accuracy_score', 'balanced_accuracy_score', 'model_best_params']
data = {
    'model_name': classifier.__class__.__name__, 
    'accuracy_score': accuracy_score, 
    'balanced_accuracy_score': balanced_accuracy_score, 
    'model_best_params': [model_best_params]
}
df_accuracy = pd.DataFrame(data=data,
                            columns=columns)
ACCURACY_FILENAME = 'data/accuracy.csv'
fileexists = os.path.isfile(ACCURACY_FILENAME)
header = False
mode = 'a'
if fileexists == False:
    mode='w'
    header=True    
df_accuracy.to_csv(ACCURACY_FILENAME,mode=mode, header=header, columns=columns, index=False, sep=',')
df_accuracy

### Classification Report

In [ ]:
encoder = preprocessing.LabelEncoder()        
encoder.fit(df_train['Intencion'])    
y_test_labels = encoder.inverse_transform(y_test)
pred_labels = list(encoder.inverse_transform(pred))
report = Accuracy.get_classification_report(y_test_labels, pred_labels)

### Save predicted test to CSV

In [ ]:
pred_det = optimized_model.predict(df_test['Pregunta'])
df_test['Intencion'] = pred_det

df_test['Intencion_cat_label'] = encoder.inverse_transform(df_test['Intencion'])
df_test['Intencion_cat'] = df_test['Intencion_cat_label'].str[4:]
SUBMIT_FILE = 'data/submit_{}.csv'.format(classifier.__class__.__name__)
df_test.to_csv(SUBMIT_FILE,mode='w', header=False, columns=['id','Intencion_cat'], index=False, sep=',')

df_test.sample(3)